# Encuesta Nacional de la Dinámica Demógrafica.



##  Diccionario de Datos 

Como primer objetivo, vamos a extraer los diccionarios de datos.

### Descarga
Para la descarga vamos a recuperar el url del archivo. Primero descargaremos las librerías correspondientes y haremos la conexión.

In [82]:

"""
import requests as r
from bs4 import BeautifulSoup

enadid_url = "https://www.inegi.org.mx/programas/enadid/2018/"

tabs = ['documentacion','tabulados','microdatos', 'datos_abiertos','herramientas']

url = enadid_url + "#" + tabs[2]

response = r.get(url)

"""


'\nimport requests as r\nfrom bs4 import BeautifulSoup\n\nenadid_url = "https://www.inegi.org.mx/programas/enadid/2018/"\n\ntabs = [\'documentacion\',\'tabulados\',\'microdatos\', \'datos_abiertos\',\'herramientas\']\n\nurl = enadid_url + "#" + tabs[2]\n\nresponse = r.get(url)\n\n'

Una vez la conexión esta hecha. Entonces partimos a extraer el texto.

In [83]:
import numpy as np
import pandas as pd

enadid = pd.read_excel('fd_enadid_2018_2014_2018.xlsx', sheet_name=None)
dictsheets = enadid.keys()
dictsheets

dict_keys(['TVIVIENDA', 'THOGAR', 'TSDEM', 'TMIGRANTE', 'TFECHISEMB', 'TMUJER1', 'TMUJER2'])

## Analísis
Una vez hayamos logrado descargar el diccionario de datos. Nuestro objetivo es crear un documento que almacene la mayor cantidad de variables posibles. Además vimos que un diccionario posee varias hojas. Entonces partiremos de la primera hoja. 

In [94]:
test_analysis = enadid['TMUJER1']
test_analysis.head()

,Unnamed: 0,INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34,ESTRUCTURA DEL ARCHIVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,Encuesta Nacional de la Dinámica Demográfica 2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34,Prueba estadística ENADID 2014-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34,Tabla de datos: ENADID_2018-2014.TMUJER1_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vemos varias cosas que no nos interesan. Por ejemplo, el texto que aparece en los primeros renglones.

In [95]:
def t_header():
    indicate_remove = test_analysis[
    'INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].isna()
    indicate_remove = list(indicate_remove)
    indicate_remove
    remove = 0
    t = 0
    while remove < 3:
        if indicate_remove[t] == True:
            remove += 1
        t += 1
    return t

def remove_header(t):
    remove_index = np.arange(0, t)
    test_analysis = test_analysis.drop(index = remove_index)

Ahora, eliminamos los renglones vacíos.

In [96]:
test_analysis['Unnamed: 0'] = test_analysis.isna().sum(axis = 1)
n_cols = test_analysis['Unnamed: 0'].max()
# test_analysis = test_analysis[test_analysis['Unnamed: 0'] < n_cols]

Ahora debemos seleccionar las categórias de cada una de las tablas que aparecen.

In [97]:
sub_tables = test_analysis[test_analysis['Unnamed: 0'] == (n_cols - 1)]
names = sub_tables['INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación '].to_list()
sub_index = np.array(sub_tables.index)
sub_index = sub_index[sub_index > t_header()]


subdfs = []
for i in range(len(sub_index) - 1):
    subdfs.append(test_analysis.iloc[sub_index[i]:sub_index[i + 1],:])
    if sub_index[i + 1] == sub_index[-1]:
        subdfs.append(test_analysis.iloc[sub_index[i + 1]:,:])


Ahora, que las tenemos en una lista. Procedemos a limpiarlas.

In [100]:
q = subdfs[0]
q = q.drop(columns= q.columns[0], axis = 1)
q

,INEGI. Encuesta Nacional de la Dinámica Demográfica 2014-2018. Descriptor de base de explotación,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
18,Identificación del registro de mujer con módulo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,ID,Mnemónico_PE,Nombre,Descripción,Tipo,Long.,Códigos válidos,Clases,NaN,Catálogo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Valor,Etiqueta,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,(0),(1),(2),(3),(4),(5),(6),(7),NaN,(8),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,1,UPM,Unidad Primaria de Muestreo,Unidad Primaria de Muestreo,Caracter,7,0100001...3260039,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2,VIV_SEL,Vivienda seleccionada,Vivienda seleccionada,Caracter,2,01..10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,3,HOGAR,Consecutivo de hogar,Consecutivo del hogar en la vivienda,Caracter,1,1….5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,4,N_REN,Número de renglón,Consecutivo del residente habitual miembro del...,Caracter,2,01…25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:

def df_block(ldf):
    superdf = pd.DataFrame()
    for q, df_test in enumerate(ldf):
        print(q)
        df_test = subdfs[q]
        df_test.columns = df_test.iloc[1,:]
        df_name = df_test.iloc[0, 1]
        if q == 0:
            df_test = df_test.drop(columns=int(2)).iloc[4: ,:]
        else:
            df_test = df_test.drop(columns=int(2)).iloc[3: ,:]
        df_clean = df_test.dropna(axis= 0, how='all').ffill()
        df_clean['SubNombre'] = df_name
        superdf = pd.concat([superdf, df_clean])
    return superdf

# dfls = df_block(subdfs)


Finalmete, lo guardamos en un excel llamado . result.xlsx.

In [90]:
# dfls.set_index('ID',inplace=True)
# dfls.to_excel('result_test5.xlsx')

De las pruebas, vimos que hay problemas cuando cambian las columnas. Entonces lo que haremos será asegurarnos que se mantenga uniforme. Primero definimos la 

In [91]:
def addtext_tolist(lista: list, texto: str,lado: str):
    for i in range(len(lista)):
        if lado == 'right':
            lista[i] = lista[i] + texto
        else:
            lista[i] = texto + lista[i]
    return lista

def comparelists(listaA: list, listaB: list):
    d_index = []
    if len(listaA) == len(listaB):
        for i in range(len(listaA)):
            if listaA[i] != listaB[i]:
                d_index.append(i)
    return d_index


Ahora si, modificaremos el código anterior para así poder leer cualquier base.

In [92]:
def df_blocks(ldf):
    superdf = pd.DataFrame()
    for q, df_test in enumerate(ldf):
        df_test.columns = df_test.iloc[1,:]
        df_name = df_test.iloc[0, 1]
        if q == 0:
            df_test = df_test.drop(columns=int(2)).iloc[4: ,:]
        else:
            sdfcols = list(superdf.columns)
            df_columns = list(df_test.columns)
            modcols = comparelists(sdfcols, df_columns)
            if len(modcols) > 0:
                for j in modcols:
                    new_name = sdfcols[j]
                    last_name = df_columns[j]
                    df_test = df_test.rename(columns = {last_name: new_name})
                    modific = list(df_test[last_name])
                    modific = addtext_tolist(modific,last_name + ": ")
                    df_test[last_name] = modific
            df_test = df_test.drop(columns=int(2)).iloc[3: ,:]
        df_clean = df_test.dropna(axis= 0, how='all').ffill()
        df_clean['SubNombre'] = df_name
        superdf = pd.concat([superdf, df_clean])
    return superdf


In [93]:
# df_block(subdfs)